# Module 1: Poisson equation on a unit square #

In this first part of the tutorial, we will solve the Poisson equation, using Dirichlet Dirichlet boundary conditions. The PDE is
$$
-\Delta u = f \text{ on } \Omega, \quad u=u_D \text{ on } \partial \Omega
$$
In this first example, $\Omega = [0,1]\times[0,1]$, the unit square.. The function $f$ and the boundary conditions are chosen such that the exact solution is $u_e(x,y) = 1+x^2+2y^2$. 

In order to solve this problem numerically, using FENiCSx, the following steps has to be taken:
1. Import all required modules and functions.
2. Define the domain, and the discrete function space defined on the domain.
3. Locate the boundary dofs, and set the boundary conditions.
4. Set up the complete discrete variational problem.
5. Solve the linear system of equations. 
6. Visualize the solution.

This is an adaption of the Jupyter notebook *dolfinx/chapter1/Fundamentals* in the example folder. This explains each function in more details. 

#### 1. Import all the required modules and functions ####

In [ ]:
from mpi4py import MPI
import pyvista
pyvista.set_jupyter_backend('static')
import numpy as np
from dolfinx import default_scalar_type
from dolfinx.fem import (
    Constant,
    Function,
    functionspace,
    dirichletbc,
    locate_dofs_topological
)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.mesh import (
    create_unit_square, 
    exterior_facet_indices, 
    CellType, locate_entities_boundary
    )
from dolfinx.plot import vtk_mesh
from dolfinx.io import XDMFFile
from ufl import (
    SpatialCoordinate, 
    TestFunction, TrialFunction, 
    dot, ds, dx, grad
)

#### 2. Create the domain (a unit square), and the discrete function space. ####

In [ ]:
domain = create_unit_square(MPI.COMM_WORLD, 8, 8, CellType.quadrilateral)
V = functionspace(domain, ("Lagrange",1))

Define a function on the discrete space $V$, and set it to the exact solution. 

This be used set the boundary conditions later. 

In [ ]:
uD = Function(V)
uD.interpolate(lambda x: 1+x[0]**2+2*x[1]**2)

#### 3. Locate the boundary dofs, and set the boundary values. ####

In [ ]:
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = exterior_facet_indices(domain.topology)
boundary_dofs = locate_dofs_topological(V, fdim, boundary_facets)
bc = dirichletbc(uD, boundary_dofs)

#### 4. Set up the variational form. ####
The weak formulation is
$$
\text{Find } u \in \hat{V} \text{ such that }  a(u,v)=L(v) \text{ for all } v\in V 
$$
with $a(u,v) = \int_{\Omega}\nabla u\cdot \nabla v dx$ and $L(v) = \int_{\Omega}fv dx$. 

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(domain, default_scalar_type(-6))
a = dot(grad(u), grad(v))*dx
L = f*v*dx

#### 5. Set up and solve the linear system. ####

In [ ]:
problem = LinearProblem(a, L, bcs=[bc],
                        petsc_options={"ksp_type": "preonly", "pc_type": "lu"},
                        petsc_options_prefix="Poisson2D_"
                      )
uh = problem.solve()

#### 5. Visualization. ####

First som preparations. 

In [ ]:
domain.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

Plot of the grid

In [ ]:
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges = True)
plotter.view_xy()
plotter.show()

Show the solution as a heat map.

In [ ]:
u_topology, u_cell_types, u_geometry = vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
u_plotter.show()

Or as a surface.

In [ ]:
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
plotter2.show()

Write the mesh and the solution to an xdmf file, for further visualization with paraview.

In [ ]:
from dolfinx import io
with io.XDMFFile(domain.comm, "poisson_solution.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(uh)